In [26]:
# conda env: st(Python 3.12.4)
# conda env: clamp_env (Python 3.8.0)
import os
import sys
from datacat4ml.const import DATA_DIR, FIG_DIR, FETCH_DATA_DIR , FETCH_FIG_DIR

import pandas as pd
import twinbooster

In [27]:
# load the dataset
or_chembl_id = ['CHEMBL233', 'CHEMBL237', 'CHEMBL236', 'CHEMBL2014']
gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))
# extract the rows where the 'target_chembl_id' is one of the elements in the list OR_chembl_id
or_ki = gpcr_ki[gpcr_ki['target_chembl_id'].isin(or_chembl_id)]

/tmp/ipykernel_3756094/3583070045.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  gpcr_ki = pd.read_csv(os.path.join(DATA_DIR, 'data_prep', '1_data_fetch', 'ki_maxcur_8_data.csv'))


In [30]:
twinbooster.download_models() # download the pre-trained models. This step is necessary only once.
tb = twinbooster.TwinBooster()

Extracting LightGBM model...
Extracting Barlow Twins model...


/storage/homefs/yc24j783/miniconda3/envs/twinbooster/lib/python3.8/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/storage/homefs/yc24j783/miniconda3/envs/twinbooster/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [37]:
active_threshold, inactive_threshold = tb.thresholds
print(f'Active threshold: {active_threshold},\nInactive threshold: {inactive_threshold}')

Active threshold: 0.5918970516203166,
Inactive threshold: 0.30255694514339015


conf = confidence

`confidence = [1 if (x >= active_threshold) or (x <= inactive_threshold) else 0 for x in pred]`

pred --> predict --> predicted probabilities

`pred = self.lgbm.predict_proba(embedding)[:, 1]`

In [38]:
or_ki_mols = or_ki.canonical_smiles.to_list()
or_ki_mols

['CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](NC(=O)CNC(=O)CCC(=O)NCC(=O)N[C@H]2CC[C@@]4(O)[C@H]6Cc7ccc(O)c8c7[C@@]4(CCN6C)[C@H]2O8)CC[C@@]3(O)[C@H]1C5',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CCCCCCC(c1ccccc1)N1CC[C@H]1[C@@H](N)c1cccc(Cl)c1',
 'CN1CC[C@]23c4c5ccc(O)c4O[C@H]2[C@@H](NC(=O)CNC(=O)CNC(=O)CNC(=O)CCC(=O)NCC(=O)NCC(=O)NCC(=O)N[C@H]2CC[C@@]4(O)[C@H]6Cc7ccc(O)c8c7[C@@]4(CCN6C)[C@H]2O8)CC[C@@]3(O)[C@H]1C5',
 'O=C(CCC(=O)NCC(=O)N[C@@H]1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3CC2CC2)[C@H]1O5)NCC(=O)N[C@@H]1CC[C@@]2(O)[C@H]3Cc4ccc(O)c5c4[C@@]2(CCN3CC2CC2)[C@H]1O5',
 'Cc1ccc(C(c2ccc(C)cc2)N2CC[C@H]2[C@H](N)c2cccc(Cl)c2)cc1',
 'Cc1ccc(C(c2ccc(C)cc2)N2CC[C@H]2[C@H](N)c2cccc(Cl)c2)c

In [39]:
or_ki_assay_descs = or_ki.assay_desc.to_list()
or_ki_assay_descs

['Displacement of [3H]EK from Opioid receptor delta 1 in guinea pig brain membrane',
 'Binding affinity at human ORL1 receptor was determined by using [125I]nociceptin radioligand in Chinese hamster ovary (CHO) cell membranes at a concentration of 23 pM',
 'Binding affinity at human opioid receptor mu 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.12 nM',
 'Binding affinity at human opioid receptor kappa 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.31 nM',
 'Binding affinity at human opioid receptor delta 1 was determined by using [3H]diprenorphine radioligand in CHO cell membranes at a concentration of 0.8 nM',
 'Binding affinity at human ORL1 receptor was determined by using [125I]nociceptin radioligand in Chinese hamster ovary (CHO) cell membranes at a concentration of 23 pM',
 'Binding affinity at human opioid receptor mu 1 was determined by using [3H]diprenorphine radiol

In [40]:
molecules = or_ki_mols[:1000]
assay_descriptions = or_ki_assay_descs[0]

In [41]:
# TwinBooster/twinbooster/scripts/model.py/`class TwinBooster`/`def predict`
# pred: 
pred, conf = tb.predict(molecules, assay_descriptions, get_confidence=True) 
df = pd.DataFrame({'molecule': molecules, 'prediction': pred, 'confidence': conf})

In [45]:
pred

array([0.63984406, 0.66043077, 0.66043077, 0.66043077, 0.66043077,
       0.8218018 , 0.8218018 , 0.8218018 , 0.8218018 , 0.62797672,
       0.81641387, 0.8346686 , 0.8346686 , 0.8346686 , 0.8346686 ,
       0.71527032, 0.71527032, 0.71527032, 0.71527032, 0.85336409,
       0.79058307, 0.79058307, 0.79058307, 0.79058307, 0.75347643,
       0.75347643, 0.75347643, 0.75347643, 0.75439565, 0.76165385,
       0.76165385, 0.76165385, 0.76165385, 0.76165385, 0.76165385,
       0.76165385, 0.76165385, 0.74564433, 0.74564433, 0.74564433,
       0.74564433, 0.53516026, 0.53516026, 0.50344747, 0.63984406,
       0.74016558, 0.74016558, 0.74016558, 0.74016558, 0.69181388,
       0.69181388, 0.69181388, 0.69181388, 0.75165104, 0.59978505,
       0.64471661, 0.71814833, 0.71814833, 0.71814833, 0.71814833,
       0.66489913, 0.66489913, 0.66489913, 0.66489913, 0.77827042,
       0.77827042, 0.77827042, 0.77827042, 0.77050633, 0.77050633,
       0.77050633, 0.77050633, 0.71527032, 0.71527032, 0.71527

In [44]:
df.iloc[151:200]

,molecule,prediction,confidence
151,N[C@@H](c1cccc(F)c1)[C@@H]1CCN1C(c1ccccc1)c1cc...,0.737918,1
152,CCCC(c1ccccc1)N1CC[C@H]1[C@H](N)c1cccc(Cl)c1,0.778270,1
153,CCCC(c1ccccc1)N1CC[C@H]1[C@H](N)c1cccc(Cl)c1,0.778270,1
154,CCCC(c1ccccc1)N1CC[C@H]1[C@H](N)c1cccc(Cl)c1,0.778270,1
155,CCCC(c1ccccc1)N1CC[C@H]1[C@H](N)c1cccc(Cl)c1,0.778270,1
156,CCNC(=O)c1ccc(C(=C2CC3CCC(C2)N3c2cccn2C)c2cccc...,0.790564,1
157,CCNC(=O)c1ccc(C(=C2CC3CCC(C2)N3c2cccn2C)c2cccc...,0.790564,1
158,CC[C@H](C)[C@H](NC(=O)[C@H](CCCN=C(N)N)NC(=O)[...,0.490198,0
159,CC[C@H](C)[C@H](NC(=O)[C@H](CCCN=C(N)N)NC(=O)[...,0.490198,0
160,CC[C@H](C)[C@H](NC(=O)[C@H](CCCN=C(N)N)NC(=O)[...,0.490198,0
